Úkol 1
V souboru s daty na Slacku máš data o hodnotách finančních indikátorů 100 největších společností obchodovaných na americké burze. Naším cílem je zjistit, které indikátory nejvíce ovlivňují cenu, a vytvořit model, který odhadne cenu akcie na základě hodnot finančních indikátorů.

Jeden z indikátorů je označený jako Y a je poměrem ceny a účetní hodnoty akcie. Hodnota indikátoru je odrazem ceny akcie. Ostatní indikátory jso následující:

běžná likvidita (Current Ratio, CR),
zadluženost (Debt to Assets, DA),
finanční páka (Financial Leverage, FL)
provozní zisková marže (Operating Profit Margin, OPM),
obrat pohledávek (Receivables Turnover, RT),
obrat celkových aktiv (Total Assets Turnover, TAT).
Úkol můžeš odevzdat jako Jupyter notebook.
Nenahrávej prosím datový soubor na GitHub (neobsahuje veřejně dostupná data). Ze stejného důvodu jsou data anonymizovaná, tj. není v nich obsažen konkrétní název firmy.

In [3]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
data = pd.read_csv("df.csv")
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical


In [53]:
data.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'predictions', 'residuals', 'predictions2', 'residuals2',
       'predictions3', 'residuals3', 'Sector_Basic Materials',
       'Sector_Communication Services', 'Sector_Conglomerates',
       'Sector_Consumer Cyclical', 'Sector_Consumer Defensive',
       'Sector_Energy', 'Sector_Financial Services', 'Sector_Healthcare',
       'Sector_Industrials', 'Sector_Technology'],
      dtype='object')

In [4]:
data["Company"].unique()

array(['2Q3DU', '46JNq', '4ISOt', '5AWwu', '5hbZR', '5OS3V', '6qLRA',
       '7CbEe', '8haSA', '8kTHi', 'A4JB2', 'aI3HK', 'aI8EZ', 'alGnD',
       'AqF25', 'AUpye', 'Awtx1', 'bCSzy', 'BKGlA', 'BndPS', 'bOls6',
       'BV6He', 'BweH1', 'c9iAO', 'Cdr0m', 'cIPg7', 'CP7VJ', 'CwEyb',
       'E1Wrw', 'ecjSU', 'EheSd', 'EmE4u', 'FdN7i', 'fGepV', 'fIy8l',
       'FsU9y', 'FWggO', 'GCagV', 'GRsRd', 'hqTVU', 'HUZTC', 'Hzcsi',
       'i2xLB', 'IFYck', 'IYuKk', 'jhFdg', 'JhQqG', 'Ji5NX', 'K4YN2',
       'KtN8a', 'LBzK4', 'Lgfee', 'lWUzX', 'MRW1e', 'mS0GZ', 'MurJ2',
       'n2wGj', 'NH3bb', 'nPtDv', 'nr5PY', 'oh7yp', 'OhaK7', 'oiv7a',
       'Oxyxu', 'P9znL', 'PeiXT', 'PEqM7', 'pgJHw', 'pkZBl', 'PqRAQ',
       'Q0dg9', 'Q2trW', 'Q7KOA', 'qdG42', 'QFoY8', 'QiwB1', 'qzrbi',
       'r1lEY', 'rr2Oh', 'SYXLy', 'tdg9S', 'tX5FA', 'Tyyob', 'UkzJz',
       'ulbpy', 'usBNB', 'vGaDH', 'vW0pU', 'vwxyQ', 'Wjagj', 'xC5LA',
       'xWEI9', 'xzg7b', 'Y0lHb', 'y6kqD', 'yhCH3', 'YqdX2', 'yUjyz',
       'YX515', 'zCR

In [5]:
data.shape

(100, 11)

Část 1
Vytvoř korelační matici a podívej se, který ukazatel má největší vliv na indikátor Y akcie.

In [6]:
data = pd.read_csv("df.csv")
data_corr = data.drop(columns=["Company", "Sector"])
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06


In [7]:
data_corr.corr()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


největší vliv na Y akcie má indikátor FL - finanční páka (Financial Leverage, FL)

Část 2
Vytvoř regresní model, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu vlož hodnoty ostatních indikátorů jako vysvětlující proměnné.

S využitím modulu statsmodels vytvoř regresní model a zobraz si tabulku se souhrnem významů. Podívej se na hodnoty koeficientů a na výsledky testu statistické významnosti koeficientů.

In [10]:
formula = "Y ~ CR+DA+FL+OPM+RT+TAT+DE+PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           1.45e-18
Time:                        22:22:37   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.042     -21.638      -0.397
CR             1.5014      1.240      1.211      0.229      -0.961       3.964
DA            -5.2206      9.370     -0.557      0.579     -23.834      13.392
FL             3.0104      1.060      2.840      0.006       0.905       5.116
OPM           23.4665      8.076      2.906      0.005       7.425      39.508
RT            -0.0063      0.021     -0.300      0.765      -0.048       0.035
TAT            8.4254      2.351      3.584      0.001       3.756      13.095
DE            -0.7283      2.066     -0.352      0.725      -4.833       3.376
PT            -0.0644      0.096     -0.669      0.505      -0.256       0.127
==============================================================================
Omnibus:                       29.136   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.551
Skew:                           0.881   Prob(JB):                     8.88e-23
Kurtosis:                       7.611   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestav nový model bez tohoto koeficientů (případně beze všech nevýznamných koeficientů).

In [11]:
formula = "Y ~ FL+OPM+TAT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        22:22:43   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.000     -16.106      -4.732
FL             2.5281      0.208     12.149      0.000       2.115       2.941
OPM           24.8478      7.542      3.295      0.001       9.878      39.818
TAT            8.5518      2.205      3.879      0.000       4.176      12.928
==============================================================================
Omnibus:                       26.748   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.291
Skew:                           0.876   Prob(JB):                     2.71e-17
Kurtosis:                       6.904   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. 

In [17]:
data_corr["predictions"] = res.fittedvalues
data["predictions"]=data_corr["predictions"]
data_corr[["FL", "OPM", "TAT", "Y", "predictions"]].head()

,FL,OPM,TAT,Y,predictions
0,2.21,0.0770,0.40,1.95,0.502359
1,10.14,0.2037,0.36,7.26,23.356512
2,1.43,0.3055,0.72,7.78,6.944718
3,3.55,0.0708,1.80,6.17,15.708512
4,0.00,0.1677,0.93,0.00,1.701358


Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená).

In [14]:
data_corr.columns

Index(['CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT', 'predictions',
       'residuals'],
      dtype='object')

In [15]:
data.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'Sector'],
      dtype='object')

In [18]:
data_corr["residuals"] = res.resid
data["residuals"]=data_corr["residuals"]
data_sorted = data.sort_values('residuals', ascending = True)
data_sorted[["Company","FL", "OPM", "TAT", "Y", "residuals", "predictions"]].head()

,Company,FL,OPM,TAT,Y,residuals,predictions
79,SYXLy,18.64,0.0631,0.41,6.97,-34.809545,41.779545
98,YX515,25.03,0.3758,0.31,41.57,-23.278968,64.848968
6,6qLRA,13.56,-0.0415,0.36,3.06,-22.850015,25.910015
1,46JNq,10.14,0.2037,0.36,7.26,-16.096512,23.356512
56,n2wGj,3.37,0.0349,3.24,11.30,-15.376012,26.676012


Nejvíce podhodnocennou akcií je: SYXLy cena na trhu je stanovena na 6.97, predikce ukazuje 41.779545

Bonus 1: Sestav model s využitím robustní regrese. Opět proveď vyřazení koeficinetů, které nejsou statisticky významné, a sestav model pouze s významnými koeficienty. Vlož odhady cen do původních dat a opět najdi potenciálně nejvíce podhodnocenou akcii. Jde v případě robustní regese o stejnou akcii, nebo se akcie liší?

In [19]:
formula = "Y ~ CR+DA+FL+OPM+RT+TAT+DE+PT"
mod2 = smf.rlm(formula=formula, data=data_corr)
res2 = mod2.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       91
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 16 Nov 2023                                         
Time:                        22:30:59                                         
No. Iterations:                    37                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4961      3.453     -3.039      0.002     -17.265      -3.728
CR             2.0316      0.801      2.537      0.011       0.462       3.601
DA            -0.4762      6.052     -0.079      0.937     -12.338      11.386
FL             3.6694      0.685      5.360      0.000       2.328       5.011
OPM           16.3810      5.216      3.140      0.002       6.158      26.604
RT            -0.0029      0.014     -0.216      0.829      -0.029       0.024
TAT            5.2670      1.518      3.469      0.001       2.291       8.243
DE            -2.5507      1.335     -1.911      0.056      -5.166       0.065
PT            -0.0878      0.062     -1.412      0.158      -0.210       0.034
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [20]:
formula = "Y ~ CR+FL+OPM+TAT"
mod2 = smf.rlm(formula=formula, data=data_corr)
res2 = mod2.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       95
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 16 Nov 2023                                         
Time:                        22:31:46                                         
No. Iterations:                    23                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.9579      1.924     -5.176      0.000     -13.729      -6.187
CR             2.0977      0.681      3.081      0.002       0.763       3.432
FL             2.4422      0.123     19.840      0.000       2.201       2.683
OPM           13.9913      4.560      3.068      0.002       5.054      22.929
TAT            5.3858      1.290      4.176      0.000       2.858       7.913
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [22]:
data_corr["predictions2"] = res2.fittedvalues
data["predictions2"]=data_corr["predictions2"]
data_corr[["FL", "OPM", "TAT", "Y", "predictions2"]].head()

,FL,OPM,TAT,Y,predictions2
0,2.21,0.0770,0.40,1.95,1.167348
1,10.14,0.2037,0.36,7.26,20.203407
2,1.43,0.3055,0.72,7.78,7.832927
3,3.55,0.0708,1.80,6.17,11.557664
4,0.00,0.1677,0.93,0.00,-0.085471


In [25]:
data_corr["residuals2"] = res2.resid
data["residuals2"]=data_corr["residuals2"]
data_corr[["FL", "OPM", "TAT", "Y", "residuals2","predictions2"]].head()

,FL,OPM,TAT,Y,residuals2,predictions2
0,2.21,0.0770,0.40,1.95,0.782652,1.167348
1,10.14,0.2037,0.36,7.26,-12.943407,20.203407
2,1.43,0.3055,0.72,7.78,-0.052927,7.832927
3,3.55,0.0708,1.80,6.17,-5.387664,11.557664
4,0.00,0.1677,0.93,0.00,0.085471,-0.085471


In [26]:
data_sorted2 = data.sort_values('residuals2', ascending = True)
data_sorted2[["Company","FL", "OPM", "TAT", "Y", "residuals2", "predictions2"]].head()

,Company,FL,OPM,TAT,Y,residuals2,predictions2
79,SYXLy,18.64,0.0631,0.41,6.97,-33.280244,40.250244
6,6qLRA,13.56,-0.0415,0.36,3.06,-23.953046,27.013046
98,YX515,25.03,0.3758,0.31,41.57,-21.352966,62.922966
1,46JNq,10.14,0.2037,0.36,7.26,-12.943407,20.203407
24,Cdr0m,3.14,0.0187,1.97,1.91,-9.315472,11.225472


I v případě robustní regrese jde o stejnou akcii. 

In [51]:
data_sorted2.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'Sector', 'predictions', 'residuals', 'predictions2', 'residuals2'],
      dtype='object')

In [52]:
data.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'predictions', 'residuals', 'predictions2', 'residuals2',
       'predictions3', 'residuals3', 'Sector_Basic Materials',
       'Sector_Communication Services', 'Sector_Conglomerates',
       'Sector_Consumer Cyclical', 'Sector_Consumer Defensive',
       'Sector_Energy', 'Sector_Financial Services', 'Sector_Healthcare',
       'Sector_Industrials', 'Sector_Technology'],
      dtype='object')

Bonus 2: Použij původní (tedy "nerobustní") model a vyčísli Cookovu vzdálenost pro všechny hodnoty. 

In [28]:
from statsmodels.stats.outliers_influence import OLSInfluence

In [29]:
formula = "Y ~ CR+DA+FL+OPM+RT+TAT+DE+PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
influence = OLSInfluence(res)
data_corr['Cook Distance'] = influence.cooks_distance[0]
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,predictions,residuals,predictions2,residuals2,Cook Distance
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,0.502359,1.447641,1.167348,0.782652,0.000046
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,23.356512,-16.096512,20.203407,-12.943407,0.037578
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,6.944718,0.835282,7.832927,-0.052927,0.000173
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,15.708512,-9.538512,11.557664,-5.387664,0.008283
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,1.701358,-1.701358,-0.085471,0.085471,0.000082


Vyřaď všechny akcie s Cookovou vzdáleností vyšší než 1 a přepočítej regresní model. 

In [35]:
data_corr = data_corr[data_corr['Cook Distance'] <= 1]
data_corr.head()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,predictions,residuals,predictions2,residuals2,Cook Distance
0,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,0.502359,1.447641,1.167348,0.782652,0.000033
1,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,23.356512,-16.096512,20.203407,-12.943407,0.362474
2,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,6.944718,0.835282,7.832927,-0.052927,0.001029
3,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,15.708512,-9.538512,11.557664,-5.387664,0.007548
4,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,1.701358,-1.701358,-0.085471,0.085471,0.000075


In [36]:
formula = "Y ~ CR+DA+FL+OPM+RT+TAT+DE+PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     24.76
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           1.06e-19
Time:                        22:47:44   Log-Likelihood:                -367.74
No. Observations:                  99   AIC:                             753.5
Df Residuals:                      90   BIC:                             776.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.0524      5.364     -1.128      0.262     -16.708       4.604
CR             1.4779      1.186      1.246      0.216      -0.878       3.834
DA           -16.4346      9.678     -1.698      0.093     -35.661       2.792
FL             0.8796      1.228      0.716      0.476      -1.561       3.320
OPM           22.9074      7.728      2.964      0.004       7.555      38.260
RT            -0.0003      0.020     -0.013      0.990      -0.040       0.040
TAT            7.9619      2.254      3.533      0.001       3.484      12.440
DE             4.2726      2.560      1.669      0.099      -0.814       9.359
PT            -0.0192      0.093     -0.206      0.837      -0.204       0.166
==============================================================================
Omnibus:                       20.217   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.822
Skew:                           0.649   Prob(JB):                     2.05e-12
Kurtosis:                       6.371   Cond. No.                         640.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Nakonec opět najdi potenciálně nejvíce podhodnocenou akcii a podívej se, jestli jde o stejnou akcii jako u předchozích modelů.

In [37]:
formula = "Y ~ OPM+TAT"
mod3 = smf.ols(formula=formula, data=data_corr)
res3 = mod3.fit()
res3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     6.558
Date:                Thu, 16 Nov 2023   Prob (F-statistic):            0.00214
Time:                        22:50:29   Log-Likelihood:                -418.99
No. Observations:                  99   AIC:                             844.0
Df Residuals:                      96   BIC:                             851.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8327      4.303     -0.194      0.847      -9.374       7.709
OPM           16.3267     11.820      1.381      0.170      -7.137      39.790
TAT           12.3435      3.424      3.605      0.000       5.548      19.139
==============================================================================
Omnibus:                       93.205   Durbin-Watson:                   1.893
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              867.514
Skew:                           3.126   Prob(JB):                    4.19e-189
Kurtosis:                      16.085   Cond. No.                         9.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
data_corr["predictions3"] = res3.fittedvalues
data["predictions3"]=data_corr["predictions3"]
data_corr[[ "OPM", "TAT", "Y", "predictions3"]].head()

,OPM,TAT,Y,predictions3
0,0.0770,0.40,1.95,5.361863
1,0.2037,0.36,7.26,6.936713
2,0.3055,0.72,7.78,13.042446
3,0.0708,1.80,6.17,22.541602
4,0.1677,0.93,0.00,13.384773


In [39]:
data_corr["residuals3"] = res3.resid
data["residuals3"]=data_corr["residuals3"]
data_corr[["OPM", "TAT", "Y", "residuals3","predictions3"]].head()

,OPM,TAT,Y,residuals3,predictions3
0,0.0770,0.40,1.95,-3.411863,5.361863
1,0.2037,0.36,7.26,0.323287,6.936713
2,0.3055,0.72,7.78,-5.262446,13.042446
3,0.0708,1.80,6.17,-16.371602,22.541602
4,0.1677,0.93,0.00,-13.384773,13.384773


In [54]:
data_sorted3 = data.sort_values('residuals3', ascending = True)
data_sorted3[["Company", "OPM", "TAT", "Y", "residuals3", "predictions3"]].head()

,Company,OPM,TAT,Y,residuals3,predictions3
56,n2wGj,0.0349,3.24,11.30,-28.430180,39.730180
26,CP7VJ,0.1578,1.99,0.00,-26.307298,26.307298
31,EmE4u,0.0406,2.20,4.59,-22.395954,26.985954
24,Cdr0m,0.0187,1.97,1.91,-21.879384,23.789384
5,5OS3V,0.1923,1.16,0.00,-16.625425,16.625425


In [ ]:
U tohoto modelu je nejvíce podhodnocenou akcií n2wGj	0.0349	3.24	11.30	-28.430180	39.730180

Bonus 3: Přidej Sektor (poslední sloupec) jako vysvětlující proměnnou s využitím One Hot Encoding. Podívej se, které sektory mají kladný koeficient a které sektory mají záporný koeficient.

--metodu One Hot Encoding jsem si ještě neprocházela a zatím postup nechápu, takže bonus 3 vynechávám